In [1]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, StringIndexer
from pyspark.sql.functions import length, size, lit
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
##dataframe = sqlContext.read.format('csv').options(header='true', inferScheme='true').load('C:/Users/jazmi/OneDrive/Documents/DataMate/db/all.csv')
df = spark.table("all_csv")

df.columns

Out[ 2 ]: ['_c0', 'username', 'Gender', 'Education', 'Ethnicity', 'about_me_text']

In [3]:
df = df.select('username', 'about_me_text', 'Gender').na.drop()
df_length = df.withColumn('length', length(df['about_me_text']))

male_female_to_num = StringIndexer(inputCol='Gender', outputCol='label')

df = male_female_to_num.fit(df_length).transform(df_length)
df.show()

+--------------+--------------------+------+------+-----+
 username| about_me_text|Gender|length|label|
+--------------+--------------------+------+------+-----+
ShortyFire1984|....................| F| 30| 0.0|
 ROXYCHIC549|'" I am not an up...| F| 1700| 0.0|
 tequilagirl69|Laid back lovable...| F| 181| 0.0|
atxredneckgirl|Tx country music ...| F| 284| 0.0|
 mel12th|I am here to look...| F| 364| 0.0|
 Andi010274|Can't get into my...| F| 49| 0.0|
 nodutch|Music business go...| F| 67| 0.0|
 lauranmr|Of all the dating...| F| 735| 0.0|
 smbd75|Likes: fun, adven...| F| 73| 0.0|
 candipie1989|Single black fema...| F| 385| 0.0|
 tif1985|I'm on here just ...| F| 309| 0.0|
 De1964|Please local only...| F| 1118| 0.0|
 Trini1971|I am a Mom to a 1...| F| 589| 0.0|
 FiaFia1213|I like to think I...| F| 352| 0.0|
 SweetJojos| Up for anything| F| 15| 0.0|
 Tawdra83|I’m a artist. I c...| F| 434| 0.0|
 LadyTexas254|not looking for a...| F| 241| 0.0|
 Vigilia79|Hi there! I'm Chr...| F| 650| 0.0|
 ayoung292|Music is life and...| F| 105| 0.0|
 Foreverfungrl| Ask me Are| F| 10| 0.0|
+--------------+--------------------+------+------+-----+
only showing top 20 rows

In [4]:
# Tokenize data
tokened_df = Tokenizer(inputCol="about_me_text", outputCol="token_text")
tt_df = tokened_df.transform(df)
tt_df = tt_df.where(size(tt_df["token_text"]) > 10)

tt_df.show()


+-----------------+--------------------+------+------+-----+--------------------+
 username| about_me_text|Gender|length|label| token_text|
+-----------------+--------------------+------+------+-----+--------------------+
 ROXYCHIC549|'" I am not an up...| F| 1700| 0.0|['", i, am, not, ...|
 tequilagirl69|Laid back lovable...| F| 181| 0.0|[laid, back, lova...|
 atxredneckgirl|Tx country music ...| F| 284| 0.0|[tx, country, mus...|
 mel12th|I am here to look...| F| 364| 0.0|[i, am, here, to,...|
 Andi010274|Can't get into my...| F| 49| 0.0|[can't, get, into...|
 lauranmr|Of all the dating...| F| 735| 0.0|[of, all, the, da...|
 candipie1989|Single black fema...| F| 385| 0.0|[single, black, f...|
 tif1985|I'm on here just ...| F| 309| 0.0|[i'm, on, here, j...|
 De1964|Please local only...| F| 1118| 0.0|[please, local, o...|
 Trini1971|I am a Mom to a 1...| F| 589| 0.0|[i, am, a, mom, t...|
 FiaFia1213|I like to think I...| F| 352| 0.0|[i, like, to, thi...|
 Tawdra83|I’m a artist. I c...| F| 434| 0.0|[i’m, a, artist.,...|
 LadyTexas254|not looking for a...| F| 241| 0.0|[not, looking, fo...|
 Vigilia79|Hi there! I'm Chr...| F| 650| 0.0|[hi, there!, i'm,...|
 ayoung292|Music is life and...| F| 105| 0.0|[music, is, life,...|
 71NOexpectations|Just testing the ...| F| 71| 0.0|[just, testing, t...|
 LadytexanInSA|Hi I am a single ...| F| 1241| 0.0|[hi, i, am, a, si...|
Wishing_on_stars1|Looking for the w...| F| 545| 0.0|[looking, for, th...|
 KarmaChris|I know my worth a...| F| 122| 0.0|[i, know, my, wor...|
 cat2018bf|I'm a mother of 4...| F| 155| 0.0|[i'm, a, mother, ...|
+-----------------+--------------------+------+------+-----+--------------------+
only showing top 20 rows

In [5]:
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
removed_frame = stop_remove.transform(tt_df)
removed_frame.show(truncate=False)

+-----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+------+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
# Run the hashing term frequency
hashing = HashingTF(inputCol='token_text', outputCol="hashedValues")

# Transform into a DF
hashed_df = hashing.transform(removed_frame)
hashed_df.show()

+-----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+
 username| about_me_text|Gender|length|label| token_text| stop_tokens| hashedValues|
+-----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+
 ROXYCHIC549|'" I am not an up...| F| 1700| 0.0|['", i, am, not, ...|['", upgraded, me...|(262144,[961,4054...|
 tequilagirl69|Laid back lovable...| F| 181| 0.0|[laid, back, lova...|[laid, back, lova...|(262144,[4333,844...|
 atxredneckgirl|Tx country music ...| F| 284| 0.0|[tx, country, mus...|[tx, country, mus...|(262144,[9639,154...|
 mel12th|I am here to look...| F| 364| 0.0|[i, am, here, to,...|[look, people, co...|(262144,[14,7401,...|
 Andi010274|Can't get into my...| F| 49| 0.0|[can't, get, into...|[get, account, st...|(262144,[6693,189...|
 lauranmr|Of all the dating...| F| 735| 0.0|[of, all, the, da...|[dating, profiles...|(262144,[1998,761...|
 candipie1989|Single black fema...| F| 385| 0.0|[single, black, f...|[single, black, f...|(262144,[8449,134...|
 tif1985|I'm on here just ...| F| 309| 0.0|[i'm, on, here, j...|[looking, friends...|(262144,[4200,576...|
 De1964|Please local only...| F| 1118| 0.0|[please, local, o...|[please, local, c...|(262144,[1536,332...|
 Trini1971|I am a Mom to a 1...| F| 589| 0.0|[i, am, a, mom, t...|[mom, 15-year-old...|(262144,[2711,869...|
 FiaFia1213|I like to think I...| F| 352| 0.0|[i, like, to, thi...|[like, think, mos...|(262144,[329,9639...|
 Tawdra83|I’m a artist. I c...| F| 434| 0.0|[i’m, a, artist.,...|[i’m, artist., co...|(262144,[2437,963...|
 LadyTexas254|not looking for a...| F| 241| 0.0|[not, looking, fo...|[looking, anythin...|(262144,[13396,16...|
 Vigilia79|Hi there! I'm Chr...| F| 650| 0.0|[hi, there!, i'm,...|[hi, there!, chri...|(262144,[456,4054...|
 ayoung292|Music is life and...| F| 105| 0.0|[music, is, life,...|[music, life, con...|(262144,[2153,262...|
 71NOexpectations|Just testing the ...| F| 71| 0.0|[just, testing, t...|[testing, waters,...|(262144,[15889,23...|
 LadytexanInSA|Hi I am a single ...| F| 1241| 0.0|[hi, i, am, a, si...|[hi, single, hisp...|(262144,[1103,243...|
Wishing_on_stars1|Looking for the w...| F| 545| 0.0|[looking, for, th...|[looking, whole, ...|(262144,[3067,104...|
 KarmaChris|I know my worth a...| F| 122| 0.0|[i, know, my, wor...|[know, worth, don...|(262144,[9639,244...|
 cat2018bf|I'm a mother of 4...| F| 155| 0.0|[i'm, a, mother, ...|[mother, 4, aweso...|(262144,[9639,158...|
+-----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+
only showing top 20 rows

In [7]:
# Fit the IDF on the data set 
idf = IDF(minDocFreq = 10, inputCol="hashedValues", outputCol="idf_features")
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)

In [8]:
# Display the DataFrame
rescaledData.show()
#rescaledData.select("token_text", "idf_features").show()

+-----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+
 username| about_me_text|Gender|length|label| token_text| stop_tokens| hashedValues| idf_features|
+-----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+
 ROXYCHIC549|'" I am not an up...| F| 1700| 0.0|['", i, am, not, ...|['", upgraded, me...|(262144,[961,4054...|(262144,[961,4054...|
 tequilagirl69|Laid back lovable...| F| 181| 0.0|[laid, back, lova...|[laid, back, lova...|(262144,[4333,844...|(262144,[4333,844...|
 atxredneckgirl|Tx country music ...| F| 284| 0.0|[tx, country, mus...|[tx, country, mus...|(262144,[9639,154...|(262144,[9639,154...|
 mel12th|I am here to look...| F| 364| 0.0|[i, am, here, to,...|[look, people, co...|(262144,[14,7401,...|(262144,[14,7401,...|
 Andi010274|Can't get into my...| F| 49| 0.0|[can't, get, into...|[get, account, st...|(262144,[6693,189...|(262144,[6693,189...|
 lauranmr|Of all the dating...| F| 735| 0.0|[of, all, the, da...|[dating, profiles...|(262144,[1998,761...|(262144,[1998,761...|
 candipie1989|Single black fema...| F| 385| 0.0|[single, black, f...|[single, black, f...|(262144,[8449,134...|(262144,[8449,134...|
 tif1985|I'm on here just ...| F| 309| 0.0|[i'm, on, here, j...|[looking, friends...|(262144,[4200,576...|(262144,[4200,576...|
 De1964|Please local only...| F| 1118| 0.0|[please, local, o...|[please, local, c...|(262144,[1536,332...|(262144,[1536,332...|
 Trini1971|I am a Mom to a 1...| F| 589| 0.0|[i, am, a, mom, t...|[mom, 15-year-old...|(262144,[2711,869...|(262144,[2711,869...|
 FiaFia1213|I like to think I...| F| 352| 0.0|[i, like, to, thi...|[like, think, mos...|(262144,[329,9639...|(262144,[329,9639...|
 Tawdra83|I’m a artist. I c...| F| 434| 0.0|[i’m, a, artist.,...|[i’m, artist., co...|(262144,[2437,963...|(262144,[2437,963...|
 LadyTexas254|not looking for a...| F| 241| 0.0|[not, looking, fo...|[looking, anythin...|(262144,[13396,16...|(262144,[13396,16...|
 Vigilia79|Hi there! I'm Chr...| F| 650| 0.0|[hi, there!, i'm,...|[hi, there!, chri...|(262144,[456,4054...|(262144,[456,4054...|
 ayoung292|Music is life and...| F| 105| 0.0|[music, is, life,...|[music, life, con...|(262144,[2153,262...|(262144,[2153,262...|
 71NOexpectations|Just testing the ...| F| 71| 0.0|[just, testing, t...|[testing, waters,...|(262144,[15889,23...|(262144,[15889,23...|
 LadytexanInSA|Hi I am a single ...| F| 1241| 0.0|[hi, i, am, a, si...|[hi, single, hisp...|(262144,[1103,243...|(262144,[1103,243...|
Wishing_on_stars1|Looking for the w...| F| 545| 0.0|[looking, for, th...|[looking, whole, ...|(262144,[3067,104...|(262144,[3067,104...|
 KarmaChris|I know my worth a...| F| 122| 0.0|[i, know, my, wor...|[know, worth, don...|(262144,[9639,244...|(262144,[9639,244...|
 cat2018bf|I'm a mother of 4...| F| 155| 0.0|[i'm, a, mother, ...|[mother, 4, aweso...|(262144,[9639,158...|(262144,[9639,158...|
+-----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [9]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_features', 'length'], outputCol='features')
cleaned_data = clean_up.transform(rescaledData)

In [10]:
cleaned_data.select(['label', 'idf_features']).show()

+-----+--------------------+
label| idf_features|
+-----+--------------------+
 0.0|(262144,[961,4054...|
 0.0|(262144,[4333,844...|
 0.0|(262144,[9639,154...|
 0.0|(262144,[14,7401,...|
 0.0|(262144,[6693,189...|
 0.0|(262144,[1998,761...|
 0.0|(262144,[8449,134...|
 0.0|(262144,[4200,576...|
 0.0|(262144,[1536,332...|
 0.0|(262144,[2711,869...|
 0.0|(262144,[329,9639...|
 0.0|(262144,[2437,963...|
 0.0|(262144,[13396,16...|
 0.0|(262144,[456,4054...|
 0.0|(262144,[2153,262...|
 0.0|(262144,[15889,23...|
 0.0|(262144,[1103,243...|
 0.0|(262144,[3067,104...|
 0.0|(262144,[9639,244...|
 0.0|(262144,[9639,158...|
+-----+--------------------+
only showing top 20 rows

In [11]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned_data.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [12]:
test_results = predictor.transform(testing)
test_results.show(5)

+----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 username| about_me_text|Gender|length|label| token_text| stop_tokens| hashedValues| idf_features| features| rawPrediction| probability|prediction|
+----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
 1KindOfLove|Hi, my name is Sh...| F| 892| 0.0|[hi,, my, name, i...|[hi,, name, sheil...|(262144,[3358,751...|(262144,[3358,751...|(262145,[9639,158...|[-2865.9850055140...|[1.0,7.6398308902...| 0.0|
1PrettyFeetChef1|BBW ... Entrepren...| F| 372| 0.0|[bbw, ..., entrep...|[bbw, ..., entrep...|(262144,[2540,705...|(262144,[2540,705...|(262145,[9639,163...|[-918.39852053098...|[1.0,1.0230260666...| 0.0|
 1cyndi|Hello. I’m new to...| F| 609| 0.0|[hello., i’m, new...|[hello., i’m, new...|(262144,[9639,133...|(262144,[9639,133...|(262145,[9639,163...|[-2065.4398811729...|[1.0,7.6271600066...| 0.0|
 2233Titans|I love sports bas...| M| 207| 1.0|[i, love, sports,...|[love, sports, ba...|(262144,[2618,127...|(262144,[2618,127...|(262145,[2618,133...|[-693.36314144946...|[0.99999999999999...| 0.0|
 2k18blessings|Committed to livi...| F| 500| 0.0|[committed, to, l...|[committed, livin...|(262144,[2099,618...|(262144,[2099,618...|(262145,[6183,163...|[-1305.0927492414...|[1.0,2.0823741158...| 0.0|
+----------------+--------------------+------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [13]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.351525